In [ ]:
pip install opendatasets

In [ ]:
import opendatasets as od
od.download('https://www.kaggle.com/datasets/supergus/multistage-continuousflow-manufacturing-process')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: vikasverma13mp
Your Kaggle Key: ··········


100%|██████████| 1.23M/1.23M [00:00<00:00, 72.8MB/s]

In [ ]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/content/multistage-continuousflow-manufacturing-process'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/content/multistage-continuousflow-manufacturing-process/notes_on_dataset.txt
/content/multistage-continuousflow-manufacturing-process/continuous_factory_process.csv


In [ ]:
from datetime import datetime
import csv
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import make_pipeline
import time
import warnings



In [ ]:
X=pd.read_csv("X.csv")
Y=pd.read_csv("Y.csv")
X2=pd.read_csv("X2.csv")
Y2=pd.read_csv("Y2.csv")

In [ ]:
for i in range (len(Y.columns[:])):
    YZ=Y.values[:,i]
    YZN=(((np.count_nonzero(YZ==0))/14088))
    if (YZN>0.2):
        YDROP=(Y.columns[i])
        print ("    Columns No  ", i, "Name  ", YDROP, "   Value =", "{:,.2f}".format(YZN))

#----- droping the columns with more than 30% zeros
DROP_Y=[1,5,6,7,11,14]
DROP_Y_N=list(Y.columns[DROP_Y])
Y=Y.drop(columns=DROP_Y_N, axis=1)


for i in range (len(Y2.columns[:])):
    Y2Z=Y2.values[:,i]
    Y2ZN=(((np.count_nonzero(Y2Z==0))/14088))
    if (Y2ZN>0.3):
        Y2DROP=(Y2.columns[i])
        print ("    Columns No  ", i, "Name  ", Y2DROP, "   Value =", "{:,.2f}".format(Y2ZN))

DROP_Y2=[4]
DROP_Y2_N=list(Y2.columns[DROP_Y2])
Y2=Y2.drop(columns=DROP_Y2_N, axis=1)

    Columns No   1 Name   Stage1.Output.Measurement1.U.Actual    Value = 0.42
    Columns No   5 Name   Stage1.Output.Measurement5.U.Actual    Value = 0.95
    Columns No   6 Name   Stage1.Output.Measurement6.U.Actual    Value = 0.33
    Columns No   7 Name   Stage1.Output.Measurement7.U.Actual    Value = 0.62
    Columns No   11 Name   Stage1.Output.Measurement11.U.Actual    Value = 0.74
    Columns No   12 Name   Stage1.Output.Measurement12.U.Actual    Value = 0.23
    Columns No   14 Name   Stage1.Output.Measurement14.U.Actual    Value = 0.36
    Columns No   4 Name   Stage2.Output.Measurement4.U.Actual    Value = 0.91


In [ ]:
input_X=X.values[:,:]
input_XC=X.columns[:]
input_XX=pd.DataFrame(data=input_X, columns=input_XC)
input_XXD=pd.DataFrame(data=input_X)

output_Y=Y.values[:,:]
output_YC=Y.columns[:]
output_YY=pd.DataFrame(data=output_Y, columns=output_YC)


In [ ]:
print("For the 1st Stage")

for i in range (len(output_YC)):
    Yi=output_YY.values [:,i];
    columns=list(input_XXD.columns)
    pmax=1
    while (len(columns)>0):
        pvalue=[]
        X_1=input_XXD[columns]
        X_1=sm.add_constant(X_1)
        model=sm.OLS(Yi, X_1).fit()
        pvalue=pd.Series(model.pvalues.values[1:], index=columns)
        pmax=max(pvalue)
        X_with_pmax=pvalue.idxmax()
        if (pmax>0.05):
            columns.remove(X_with_pmax)
        else:
                break
        selected_X=columns


    X_selected=input_XX.values[:, selected_X]

    X_train, X_test, Y_train, Y_test= train_test_split(X_selected, Yi, test_size=0.2)


#----- Implementing SVM-POLY

    svr_poly = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, kernel='poly', degree=3))
    svr_poly = svr_poly.fit(X_train, Y_train)
    svr_poly = abs(svr_poly.score(X_test, Y_test))
    svr_poly = "{:.2f}".format(svr_poly * 100)

    print(svr_poly)

For the 1st Stage
7.67
20.62
4.83
8.32
20.16
42.05
2.73
58.60
62.16


In [ ]:
print("For the 2nd Stage")


input_X2=X2.values[:,:]
input_XC2=X2.columns[:]
input_XX2=pd.DataFrame(data=input_X2, columns=input_XC2)
input_XXD2=pd.DataFrame(data=input_X2)

output_Y2=Y2.values[:,:]
output_YC2=Y2.columns[:]
output_YY2=pd.DataFrame(data=output_Y2, columns=output_YC2)


for i in range (len(output_YC2)):
    Yi2=output_YY2.values [:,i];
    columns2=list(input_XXD2.columns)
    pmax2=1
    while (len(columns2)>0):
        pvalue2=[]
        X_12=input_XXD2[columns2]
        X_12=sm.add_constant(X_12)
        model=sm.OLS(Yi2, X_12).fit()
        pvalue2=pd.Series(model.pvalues.values[1:], index=columns2)
        pmax2=max(pvalue2)
        X_with_pmax2=pvalue2.idxmax()
        if (pmax2>0.05):
            columns2.remove(X_with_pmax2)
        else:
                break
        selected_X2=columns2


    X_selected2=input_XX2.values[:, selected_X2]

    X_train2, X_test2, Y_train2, Y_test2= train_test_split(X_selected2, Yi2, test_size=0.2)


#----- Implementing SVM-POLY for the second stage

    svr_poly = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2, kernel='poly', degree=3))
    svr_poly = svr_poly.fit(X_train2, Y_train2)
    svr_poly = abs(svr_poly.score(X_test2, Y_test2))
    svr_poly = "{:.2f}".format(svr_poly * 100)

    print(svr_poly)

For the 2nd Stage
43.31
51.99
72.85
39.40
79.72
33.78
28.60
50.61
62.20
69.16
6.86
74.39
74.10
57.85
